In [28]:
# https://stackoverflow.com/questions/34322471/sqlalchemy-engine-connection-and-session-difference

In [29]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options


from bs4 import BeautifulSoup
import time
import pandas as pd
from pandas import DataFrame as DF
from IPython.display import display

from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship,backref


import smtplib
from email.mime.text import MIMEText


from urllib.request import urlopen
import json 

pd.set_option('display.max_colwidth', 60)


In [30]:
def get_browser():
    options = Options()
    options.headless = True
    _browser_profile = webdriver.FirefoxProfile()
    _browser_profile.set_preference("dom.webnotifications.enabled", False)
    executable_path='/media/disk3/feynman52/dummy/SQLAlchemy/591/geckodriver'
    browser = webdriver.Firefox(executable_path=executable_path,
                                options=options,
                                firefox_profile=_browser_profile)
    return browser

def get_page(browser,page=''):
    browser.get(page)
    time.sleep(1)
    
def click_button(browser,xpath=''):
    dummy=browser.find_element_by_xpath(xpath)
    dummy.click()
    time.sleep(1)
    
def send_keys(browser,xpath='',value=''):
    dummy=browser.find_element_by_xpath(xpath)
    dummy.send_keys(value)
    time.sleep(1)
    
def get_session(engine):
    Session = sessionmaker(bind = engine)
    session = Session()
    return session
    
def add_to_table(session,item):
    session.add(item)
    session.commit()
    
def delete_from_table(session,item):
    session.delete(item)
    session.commit()
    
def sql_to_df(engine,query=''):
    df = pd.read_sql_query(query, engine)
    return df

def get_engine(db_name=''):
    db_dir='/media/disk3/feynman52/See26/crawler-ryh'
    db_url = 'sqlite:///'+db_dir+'/'+db_name
    engine = create_engine(db_url, echo = False)
    return engine
    

def get_old_urls(session):
    old_url_objects=session.query(Urls).all()
    return old_url_objects



# xs = session.query(Push).filter(Push.article_id==None)
# for x in xs:
#     delete_from_table(session,x)

In [31]:
def get_all_urls():
    db_dir='/media/disk3/feynman52/See26/crawler-ryh'
    db_name='urls.db'
    db_url = 'sqlite:///'+db_dir+'/'+db_name
    engine_url = create_engine(db_url, echo = False)
    query='''
    select * from urls
    '''
    df=sql_to_df(engine_url,query=query)
    return df



# get articles

In [32]:
engine_article = get_engine(db_name='articles.db')

Base = declarative_base()
class Article(Base):
    __tablename__ = 'article'
    article_id = Column(Integer, primary_key=True)
    article_user = Column(String)
    article_group = Column(String)
    article_title = Column(String)
    article_time = Column(String)
    article_weekday = Column(Integer)
    article_ip = Column(String)   
    article_url = Column(String)    
    
    

class Push(Base):
    __tablename__ = 'push'
    push_id = Column(Integer, primary_key=True)
    push_user = Column(String)
    push_tag = Column(String)
    push_time = Column(String)
    push_content = Column(String)
    push_weekday = Column(Integer)
    push_ip = Column(String)
    
    article_id = Column(Integer, ForeignKey('article.article_id'))
    article = relationship("Article", back_populates = "push")
    
Article.push=relationship("Push", order_by = Push.push_id, back_populates = "article")
Base.metadata.create_all(engine_article)

# Push.__table__.drop(engine_article)
# Article.__table__.drop(engine_article)


In [33]:
db_dir='/media/disk3/feynman52/See26/crawler-ryh'
db_name='articles.db'
db_url = 'sqlite:///'+db_dir+'/'+db_name
engine_article = create_engine(db_url, echo = False)

def get_downloaded_articles_pushs(engine_article):
    query='''
    select * from article
    '''
    articles=sql_to_df(engine_article,query=query)
    
    query='''
    select * from push
    '''
    pushs=sql_to_df(engine_article,query=query)
    return articles,pushs
    


In [34]:
def append_txt_to_file(txt=''):
    with open("error.txt", "a") as myfile:
        myfile.write("%s\n"%(txt))
        
# append_txt_to_file(txt='www.333')

In [35]:
def get_current_page(browser,url):
    get_page(browser,page=url)
    pageSource = browser.page_source  
    current_page = BeautifulSoup(pageSource)  
    return current_page

In [36]:
def get_article(current_page,article_url):
    article_meta_value=current_page.find_all('span',{'class':'article-meta-value'})
    article_meta_value=[x.string for x in article_meta_value]
    article_user=article_meta_value[0].split(' ')[0]
    article_group=article_meta_value[1]
    article_title=article_meta_value[2]
    article_time=str(pd.to_datetime(article_meta_value[3]))
    article_weekday=pd.to_datetime(article_time).weekday()
    article_ip=current_page.find_all('span',{'class':'f2'})
    article_ip=[x.string for x in article_ip if x.string!=None and '來自' in x.string]
    
    try: 
        article_ip=article_ip[0]
        article_ip=article_ip[article_ip.find('來自: ')+len('來自: '):]
        article_ip=article_ip.split(' ')[0].replace('\n','')
    except:
        article_ip='no_article_ip'
    

    article=Article(
        article_user = article_user,
        article_group = article_group,
        article_title = article_title,
        article_time = article_time,
        article_weekday = article_weekday,
        article_ip = article_ip,
        article_url = article_url
    )
    
    return article


# get pushs

In [37]:
def get_pushs(current_page,article_time):
    pushs=current_page.find_all('div',{'class':'push'})
    push_objects=[]
    for push in pushs:
        push=push.find_all('span')
        
        if len(push)==0: continue
        
        push=[x.text.replace('\n','') for x in push]
        push_tag=push[0].replace(' ','')

        push_user=push[1]

        push_content=push[2].replace(': ','')

        push_ipdt=push[3]

        push_ip=push_ipdt[:-len('07/23 15:53')].replace(' ','')
        push_ip=push_ip if len(push_ip)>0 else 'no_push_ip'

        push_time=' '.join([article_time[:4],push_ipdt[-len('07/23 15:53'):]])
        push_time=str(pd.to_datetime(push_time))

        push_weekday=pd.to_datetime(push_time).weekday()

        push=Push(
            push_user = push_user,
            push_tag = push_tag,
            push_time = push_time,
            push_content = push_content,
            push_weekday = push_weekday,
            push_ip = push_ip
        )
        push_objects.append(push)
        
    return push_objects
    

In [38]:
browser=get_browser()
get_page(browser,page='https://www.ptt.cc/bbs/Gossiping/index.html')
click_button(browser,xpath='/html/body/div[2]/form/div[1]/button')
session=get_session(engine_article)

all_urls=get_all_urls()

In [64]:
all_urls.head(1)

,url_id,url_title,url_link,url_user,url_hot
0,1,第一耶！,https://www.ptt.cc/bbs/Gossiping/M.1119222611.A.7A9.html,gogoegg,74


In [92]:
c1=all_urls.url_title.apply(lambda x: '空姐' in x )
c2=all_urls.url_title.apply(lambda x: '長榮' in x )
c= c1 & c2
select_urls=all_urls[c]

In [93]:
select_urls=select_urls.url_link.tolist()


In [94]:

old_articles,old_pushs=get_downloaded_articles_pushs(engine_article)
old_urls=old_articles.article_url.tolist()


In [95]:
select_remove_old_urls=[x for x in select_urls if x not in old_urls]
len(select_urls),len(select_remove_old_urls),len(old_urls)

(504, 567, 65)

In [96]:

for i,url in enumerate(select_remove_old_urls):
    print('---------------')
    print('%s/%s'%(i+1,len(select_remove_old_urls)))
    print(url)
    
    try:
        current_page=get_current_page(browser,url=url)
    
        article=get_article(current_page,url)
        pushs=get_pushs(current_page,article_time=article.article_time)
        article.push = pushs

        add_to_table(session,article)
        
    except Exception as e:
        print(e)
        append_txt_to_file(txt='---------------')
        append_txt_to_file(txt=url)
        append_txt_to_file(txt=e)
    

---------------
1/504
https://www.ptt.cc/bbs/Gossiping/M.1559662279.A.BB6.html
('Unknown string format:', '2019  06/04 23:')
---------------
2/504
https://www.ptt.cc/bbs/Gossiping/M.1559663978.A.308.html
---------------
3/504
https://www.ptt.cc/bbs/Gossiping/M.1559724526.A.11B.html
---------------
4/504
https://www.ptt.cc/bbs/Gossiping/M.1559904856.A.3A2.html
---------------
5/504
https://www.ptt.cc/bbs/Gossiping/M.1560174281.A.D04.html
---------------
6/504
https://www.ptt.cc/bbs/Gossiping/M.1560175338.A.577.html
---------------
7/504
https://www.ptt.cc/bbs/Gossiping/M.1561021087.A.537.html
---------------
8/504
https://www.ptt.cc/bbs/Gossiping/M.1561021487.A.434.html
---------------
9/504
https://www.ptt.cc/bbs/Gossiping/M.1561038037.A.18B.html
---------------
10/504
https://www.ptt.cc/bbs/Gossiping/M.1561039229.A.30C.html
---------------
11/504
https://www.ptt.cc/bbs/Gossiping/M.1561043556.A.B7E.html
---------------
12/504
https://www.ptt.cc/bbs/Gossiping/M.1561044618.A.6C7.html
---

---------------
103/504
https://www.ptt.cc/bbs/Gossiping/M.1561284578.A.2F6.html
---------------
104/504
https://www.ptt.cc/bbs/Gossiping/M.1561293630.A.CEC.html
---------------
105/504
https://www.ptt.cc/bbs/Gossiping/M.1561305885.A.866.html
---------------
106/504
https://www.ptt.cc/bbs/Gossiping/M.1561305935.A.261.html
---------------
107/504
https://www.ptt.cc/bbs/Gossiping/M.1561306985.A.495.html
---------------
108/504
https://www.ptt.cc/bbs/Gossiping/M.1561307132.A.DF7.html
---------------
109/504
https://www.ptt.cc/bbs/Gossiping/M.1561307560.A.C1B.html
---------------
110/504
https://www.ptt.cc/bbs/Gossiping/M.1561308389.A.87D.html
---------------
111/504
https://www.ptt.cc/bbs/Gossiping/M.1561309695.A.E89.html
---------------
112/504
https://www.ptt.cc/bbs/Gossiping/M.1561309831.A.4D0.html
---------------
113/504
https://www.ptt.cc/bbs/Gossiping/M.1561313452.A.04E.html
---------------
114/504
https://www.ptt.cc/bbs/Gossiping/M.1561316768.A.BF6.html
---------------
115/504
http

---------------
205/504
https://www.ptt.cc/bbs/Gossiping/M.1561449390.A.AF8.html
---------------
206/504
https://www.ptt.cc/bbs/Gossiping/M.1561451075.A.C5C.html
---------------
207/504
https://www.ptt.cc/bbs/Gossiping/M.1561451719.A.08D.html
---------------
208/504
https://www.ptt.cc/bbs/Gossiping/M.1561451721.A.E66.html
---------------
209/504
https://www.ptt.cc/bbs/Gossiping/M.1561452526.A.B41.html
---------------
210/504
https://www.ptt.cc/bbs/Gossiping/M.1561452585.A.1D9.html
---------------
211/504
https://www.ptt.cc/bbs/Gossiping/M.1561452723.A.F3C.html
---------------
212/504
https://www.ptt.cc/bbs/Gossiping/M.1561453719.A.6EF.html
---------------
213/504
https://www.ptt.cc/bbs/Gossiping/M.1561454342.A.391.html
---------------
214/504
https://www.ptt.cc/bbs/Gossiping/M.1561454386.A.3A9.html
---------------
215/504
https://www.ptt.cc/bbs/Gossiping/M.1561454699.A.EE1.html
---------------
216/504
https://www.ptt.cc/bbs/Gossiping/M.1561454730.A.F49.html
---------------
217/504
http

---------------
307/504
https://www.ptt.cc/bbs/Gossiping/M.1561629041.A.901.html
---------------
308/504
https://www.ptt.cc/bbs/Gossiping/M.1561630363.A.E2B.html
---------------
309/504
https://www.ptt.cc/bbs/Gossiping/M.1561637708.A.132.html
---------------
310/504
https://www.ptt.cc/bbs/Gossiping/M.1561639427.A.4C7.html
---------------
311/504
https://www.ptt.cc/bbs/Gossiping/M.1561645411.A.D22.html
---------------
312/504
https://www.ptt.cc/bbs/Gossiping/M.1561645941.A.9E4.html
---------------
313/504
https://www.ptt.cc/bbs/Gossiping/M.1561646466.A.669.html
---------------
314/504
https://www.ptt.cc/bbs/Gossiping/M.1561648097.A.54C.html
---------------
315/504
https://www.ptt.cc/bbs/Gossiping/M.1561649787.A.A90.html
---------------
316/504
https://www.ptt.cc/bbs/Gossiping/M.1561651105.A.BC8.html
---------------
317/504
https://www.ptt.cc/bbs/Gossiping/M.1561661701.A.617.html
---------------
318/504
https://www.ptt.cc/bbs/Gossiping/M.1561688851.A.496.html
---------------
319/504
http

---------------
409/504
https://www.ptt.cc/bbs/Gossiping/M.1562064603.A.EE7.html
---------------
410/504
https://www.ptt.cc/bbs/Gossiping/M.1562065660.A.D6E.html
---------------
411/504
https://www.ptt.cc/bbs/Gossiping/M.1562065969.A.1C6.html
---------------
412/504
https://www.ptt.cc/bbs/Gossiping/M.1562069459.A.E25.html
---------------
413/504
https://www.ptt.cc/bbs/Gossiping/M.1562091994.A.8A4.html
---------------
414/504
https://www.ptt.cc/bbs/Gossiping/M.1562117064.A.350.html
---------------
415/504
https://www.ptt.cc/bbs/Gossiping/M.1562126592.A.B1E.html
---------------
416/504
https://www.ptt.cc/bbs/Gossiping/M.1562126830.A.9B8.html
---------------
417/504
https://www.ptt.cc/bbs/Gossiping/M.1562127218.A.CC8.html
---------------
418/504
https://www.ptt.cc/bbs/Gossiping/M.1562129512.A.BCF.html
---------------
419/504
https://www.ptt.cc/bbs/Gossiping/M.1562132819.A.0BA.html
---------------
420/504
https://www.ptt.cc/bbs/Gossiping/M.1562133742.A.985.html
---------------
421/504
http

In [99]:
a=get_downloaded_articles_pushs(engine_article)[0]
b=get_downloaded_articles_pushs(engine_article)[1]
# pd.merge(left=b, right=a, how='inner', on='article_id')

In [102]:
len(b)/len(a),len(a),len(b)

(61.96654929577465, 568, 35197)